In [1]:
import cv2 as cv
import numpy as np
import os
import sklearn
from sklearn.svm import SVC
from skimage.feature import hog
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV ,StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
MASK_PATH = './Dataset/mask/'
NO_MASK_PATH = './Dataset/no_mask/'
haarcasade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')


In [3]:
mask_data = []
no_mask_data = []
ctr = 0
ITER_LIMIT = 200
for i in os.listdir(MASK_PATH):
    path = os.path.join(MASK_PATH,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)

    faces = haarcasade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 5)
    if not len(faces) < 1:
        for face_rect in faces:
            x, y, w, h = face_rect
            face_image = img[y:y+w, x:x+h]
            resized = cv.resize(face_image,(128,128))
            mask_data.append(resized)

            ctr += 1
    if ctr >= ITER_LIMIT: break

ctr = 0
for i in os.listdir(NO_MASK_PATH):
    path = os.path.join(NO_MASK_PATH,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)

    faces = haarcasade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 5)
    if not len(faces) < 1:
        for face_rect in faces:
            x, y, w, h = face_rect
            face_image = img[y:y+w, x:x+h]
            resized = cv.resize(face_image,(128,128))
            no_mask_data.append(resized)
            ctr += 1
    if ctr >= ITER_LIMIT: break



print(len(mask_data))
print(len(no_mask_data))

200
200


In [ ]:


masked_features = []
no_masked_features = []

for i in mask_data:
    hog_feature = hog(i, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    masked_features.append(hog_feature)
    
for i in no_mask_data:
    hog_feature = hog(i, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    no_masked_features.append(hog_feature)

In [5]:
df = pd.DataFrame(masked_features)
df['label'] = 1

df1 = pd.DataFrame(no_masked_features)
df1['label'] = 0

merged = pd.concat([df,df1])
merged

,0,1,2,3,4,5,6,7,8,9,...,8091,8092,8093,8094,8095,8096,8097,8098,8099,label
0,0.262991,0.131363,0.094823,0.058470,0.139948,0.053458,0.075796,0.175023,0.173094,0.262991,...,0.189991,0.081008,0.012316,0.020131,0.107529,0.004294,0.037123,0.083799,0.027978,1
1,0.130275,0.036271,0.095504,0.161355,0.255607,0.213633,0.255607,0.255607,0.217327,0.255607,...,0.251434,0.196653,0.044037,0.089077,0.064211,0.011920,0.028070,0.021478,0.084946,1
2,0.303027,0.303027,0.172897,0.058019,0.056720,0.009305,0.000000,0.021747,0.074975,0.294024,...,0.262052,0.006212,0.011787,0.015552,0.004167,0.018734,0.020779,0.307121,0.307121,1
3,0.134760,0.173613,0.135652,0.142662,0.178771,0.084434,0.235516,0.134965,0.227746,0.291630,...,0.245777,0.032070,0.017602,0.037729,0.245777,0.183683,0.245777,0.238576,0.217094,1
4,0.145020,0.020921,0.013232,0.046175,0.032746,0.000000,0.006616,0.000000,0.000000,0.327208,...,0.204338,0.113834,0.120246,0.085345,0.178693,0.017549,0.334519,0.069289,0.262038,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.053672,0.035987,0.005675,0.016436,0.055090,0.051869,0.079802,0.012714,0.009955,0.346398,...,0.244603,0.220293,0.142379,0.191437,0.244603,0.041537,0.220798,0.194133,0.132573,0
196,0.291585,0.000000,0.144144,0.000000,0.000000,0.000000,0.026906,0.151252,0.291585,0.291585,...,0.240347,0.273123,0.273123,0.070853,0.195918,0.000000,0.013851,0.000000,0.058958,0
197,0.042674,0.058764,0.034043,0.005509,0.036278,0.075012,0.071393,0.047238,0.126899,0.364597,...,0.288647,0.150900,0.175679,0.067926,0.129585,0.107765,0.043973,0.061925,0.150550,0
198,0.086850,0.045269,0.030479,0.019017,0.063884,0.073068,0.042838,0.034667,0.029334,0.389340,...,0.223606,0.081943,0.010049,0.195551,0.203687,0.128593,0.075757,0.194399,0.119015,0


In [6]:
y = merged['label']
x = merged.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [7]:
svm = SVC(kernel='rbf',gamma=0.01, C=0.1)
svm.fit(X_train,y_train)


cross_validation = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, cv=cross_validation,refit = True, verbose = 3) 
  
grid.fit(X_train, y_train) 


Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .........

[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:   12.1s


[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.519, total=   0.4s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.481, total=   0.4s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 127 tasks      | elapsed:   46.7s


[CV] ........ C=10, gamma=0.01, kernel=rbf, score=1.000, total=   0.4s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.962, total=   0.4s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.923, total=   0.3s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.963, total=   0.2s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.889, total=   0.2s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=1.000, total=   0.2s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=1.000, total=   0.2s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [8]:
print(grid.best_params_)

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}


In [9]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print(f"Best parameters found: {grid.best_params_}")
print(f"Best cross-validation score: {grid.best_score_}")

print(classification_report(y_test, y_pred))

Best parameters found: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation score: 0.97008547008547
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        63
           1       1.00      0.94      0.97        69

    accuracy                           0.97       132
   macro avg       0.97      0.97      0.97       132
weighted avg       0.97      0.97      0.97       132



In [10]:

matrix_kebingungan = confusion_matrix(y_pred, y_test)
print(matrix_kebingungan)

[[63  4]
 [ 0 65]]


In [45]:




img = cv.imread('./b.jpg', cv.IMREAD_GRAYSCALE)
# img = no_mask_data[-1:]
# print(img)
faces = haarcasade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 5)
if not len(faces) < 1:
    for face_rect in faces:
        x, y, w, h = face_rect
        face_image = img[y:y+w, x:x+h]
        resized = cv.resize(face_image,(128,128))

        hog_feature = hog(resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        test_data = pd.DataFrame(hog_feature).T

        result=best_model.predict(test_data)[0]
        print(result)
        if result == 0:
            result = 'No Mask'
        else:
            result = 'Mask'


        cv.putText(img, result, (100,100), cv.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 1)
        cv.imshow('',img)
        cv.waitKey(0)
        cv.destroyAllWindows()

        cv.imshow('',resized)
        cv.waitKey(0)
        cv.destroyAllWindows()

0
